In [1]:
SEED = 42
import tensorflow as tf
import pandas as pd
import numpy as np
import os
import random
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
os.environ['PYTHONHASHSEED'] = str(SEED)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
np.random.seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)

2025-05-16 19:34:09.099902: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-16 19:34:09.118035: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747416849.131110  676873 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747416849.134867  676873 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747416849.148353  676873 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
export_dir = "./exports"
X_user = np.load(export_dir + "/X_user.npy", mmap_mode='r')
X_video = np.load(export_dir + "/X_video.npy", mmap_mode='r')
y = np.load(export_dir + "/y.npy", mmap_mode='r')

In [3]:
dataset = tf.data.Dataset.from_tensor_slices(((X_user, X_video), y))
BATCH_SIZE = 2048
dataset = dataset.shuffle(10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

I0000 00:00:1747416853.412235  676873 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13470 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4080 SUPER, pci bus id: 0000:01:00.0, compute capability: 8.9


In [ ]:
from tensorflow.keras import layers, Model, Input

NUM_USER_FEATURES = 70
NUM_VIDEO_FEATURES = 72

# User Tower
user_input = Input(shape=(NUM_USER_FEATURES,), name="user_input")
user_tower = layers.Dense(256, activation='relu')(user_input)
user_tower = layers.Dense(128, activation='relu')(user_tower)
user_tower = layers.Dense(64, activation='relu')(user_tower)
user_embedding = layers.Dense(32, activation='relu')(user_tower)

# Video Tower
video_input = Input(shape=(NUM_VIDEO_FEATURES,), name="video_input")
video_tower = layers.Dense(256, activation='relu')(video_input)
video_tower = layers.Dense(128, activation='relu')(video_tower)
video_tower = layers.Dense(64, activation='relu')(video_tower)
video_embedding = layers.Dense(32, activation='relu')(video_tower)

# Merge towers
merged = layers.Concatenate()([user_embedding, video_embedding])
merged = layers.Dense(64, activation='relu')(merged)
merged = layers.Dense(32, activation='relu')(merged)
output = layers.Dense(1, activation='linear')(merged)

model = Model(inputs=[user_input, video_input], outputs=output)
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ user_input          │ (None, 70)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ video_input         │ (None, 72)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │     18,176 │ user_input[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 256)       │     18,688 │ video_input[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 128)       │     32,896 │ dense_4[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 64)        │      8,256 │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 64)        │      8,256 │ dense_5[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 32)        │      2,080 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 32)        │      2,080 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 64)        │          0 │ dense_3[0][0],    │
│ (Concatenate)       │                   │            │ dense_7[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 64)        │      4,160 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 32)        │      2,080 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_10 (Dense)    │ (None, 1)         │         33 │ dense_9[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 129,601 (506.25 KB)

 Trainable params: 129,601 (506.25 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.fit(dataset, epochs=10)

Epoch 1/10


4663/4663 ━━━━━━━━━━━━━━━━━━━━ 18s 3ms/step - loss: 0.4230 - mae: 0.4328
Epoch 2/10
4663/4663 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.4046 - mae: 0.4221
Epoch 3/10
4663/4663 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.4029 - mae: 0.4209
Epoch 4/10
4663/4663 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.4021 - mae: 0.4203
Epoch 5/10
4663/4663 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.4016 - mae: 0.4198
Epoch 6/10
4663/4663 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.4009 - mae: 0.4193
Epoch 7/10
4663/4663 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.4002 - mae: 0.4188
Epoch 8/10
4663/4663 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.3997 - mae: 0.4184
Epoch 9/10
4663/4663 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.3989 - mae: 0.4179
Epoch 10/10
4663/4663 ━━━━━━━━━━━━━━━━━━━━ 13s 3ms/step - loss: 0.3984 - mae: 0.4175


In [6]:
model.save(export_dir + "/model.keras")